In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
matplotlib.rcParams['xtick.labelsize'] = 20
matplotlib.rcParams['ytick.labelsize'] = 20
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['axes.titlesize'] = 20

matplotlib.rcParams['axes.grid'] = True
matplotlib.rcParams['grid.color'] = '0.5'
matplotlib.rcParams['grid.linewidth'] = '0.5'

matplotlib.rcParams['axes.edgecolor'] = '0.25'
matplotlib.rcParams['xtick.color'] = '0'
matplotlib.rcParams['ytick.color'] = '0'

matplotlib.rcParams['xtick.major.width'] = 2
matplotlib.rcParams['ytick.major.width'] = 2
matplotlib.rcParams['ytick.major.size'] = 10
matplotlib.rcParams['xtick.major.size'] = 10
matplotlib.rcParams['axes.spines.right'] = False
matplotlib.rcParams['axes.spines.left'] = False
matplotlib.rcParams['axes.spines.top'] = False
matplotlib.rcParams['axes.spines.bottom'] = False


matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.weight']='ultralight'
# matplotlib.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif']
matplotlib.rcParams['axes.axisbelow'] = True

In [ ]:
import glob
from matplotlib import pyplot as plt
import numpy as np
from Bio import SeqIO
from scipy import stats
import re
import random
import pandas as pd
import json

from Bio.Data import CodonTable
import itertools
from collections import Counter

In [ ]:
# plt.figure()
# plt.plot(list(range(10)), list(range(10)), color='steelblue', linestyle='', marker='o')
# plt.xlabel('howdy yall')

In [ ]:
def get_codon_dicts(n=11):
    GivenCodonTable = CodonTable.unambiguous_dna_by_id[n]
    nucleotides = ['A', 'T', 'C', 'G']
    aa_to_codons = {}
    for first in nucleotides:
        for second in nucleotides:
            for third in nucleotides:
                Codon = first + second + third
                if Codon not in CodonTable.unambiguous_dna_by_id[n].stop_codons:
                    if GivenCodonTable.forward_table[Codon] in aa_to_codons.keys():
                            aa_to_codons[GivenCodonTable.forward_table[Codon]].append(Codon)
                    else:
                        aa_to_codons[GivenCodonTable.forward_table[Codon]] = [Codon]
                else:
                    pass
    codon_to_aa = {}
    for aa, syns in aa_to_codons.items():
        for syn in syns:
            codon_to_aa[syn] = aa
    return aa_to_codons, codon_to_aa

aa_to_codon_dict, codon_to_aa_dict = get_codon_dicts()

temp = []
for aa in ['P', 'T', 'A', 'V', 'G']:
    temp.extend(aa_to_codon_dict[aa])

redundant_codons = ['CTA', 'CTG', 'CTC', 'CTT',\
                    'TCA', 'TCG', 'TCC', 'TCT',\
                    'CGA', 'CGG', 'CGC', 'CGT'] + temp


# Get the sequences for each gene for testing purposes later
Note: this requires establishing a reference genome that you will use later

In [ ]:
base_genome = '511145.12' #This is E. coli K12
seq_dicty = {}
# for infile_name in glob.glob('../Data/Orthologs/Order_Enterobacterales_Reference/mfastas_nt/*.mfasta'):
# for infile_name in glob.glob('../Data/Orthologs/Order_Enterobacterales_Representative/mfastas_nt/*.mfasta'):
for infile_name in glob.glob('../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/*.mfasta'):
#     print(infile_name)
    gene_name = infile_name.split('/')[-1].strip('.mfasta')
    records = list(SeqIO.parse(infile_name, 'fasta'))
    for record in records:
        try:
            seq_dicty[record.id][gene_name] = str(record.seq)
        except KeyError:
            seq_dicty[record.id] = {}
            seq_dicty[record.id][gene_name] = str(record.seq)
print(len(seq_dicty.keys()))

## And get dicts to identify conserved amino acid positions and consensus nucleotides

Please note that I'm considering an amino acid here as being "conserved" if 60 positions in the alignment have that same amino acid (and it's not a gap). There are 61 total positions so this just gives a little bit of flexibility to definition of conserved.

In [ ]:
conservation_dicty = {}
nt_consensus_dicty = {}
for infile_name in glob.glob('../Data/Orthologs/Order_Enterobacterales_Mixed/mfastas_nt/*.mfasta')[:]:
    gene_name = infile_name.split('/')[-1].strip('.mfasta')
    conservation_dicty[gene_name] = []
    nt_consensus_dicty[gene_name] = []
    records = list(SeqIO.parse(infile_name, 'fasta'))
    aa_seq_lol = []
    nt_seq_lol = []
    for record in records:
        aa_seq_lol.append(list(str(record.seq.translate(gap='-'))))
        nt_seq_lol.append(list(str(record.seq)))
    aa_seq_lol = np.array(aa_seq_lol)
    aa_seq_lol = aa_seq_lol.T
    for pos in aa_seq_lol:
        counter_dict = Counter(pos)
        if len(counter_dict.keys()) == 1 and counter_dict[list(counter_dict.keys())[0]] != '-':
            conservation_dicty[gene_name].append('C')
        elif len(counter_dict.keys()) == 2 and sorted(counter_dict.items(), key=lambda x: x[1])[-1][1]==60 and\
                sorted(counter_dict.items(), key=lambda x: x[1])[-1][0]!='-':
            conservation_dicty[gene_name].append('C')
        else:
            conservation_dicty[gene_name].append('V')

    nt_seq_lol = np.array(nt_seq_lol)
    nt_seq_lol = nt_seq_lol.T
    for pos in nt_seq_lol:
        counter_dict = Counter(pos)
        nt_consensus_dicty[gene_name].append(sorted(counter_dict.items(), key=lambda x: x[1])[-1][0]) 

# Read in substitution rate datasets as separate dictionaries

In [ ]:
dicty_phylop = {}
# for infile_name in glob.glob('../Results/Order_Enterobacterales_Reference/phyloP/*.wig')[:]:
# for infile_name in glob.glob('../Results/Order_Enterobacterales_Representative/phyloP/*.wig')[:]:
# for infile_name in glob.glob('../Results/Order_Enterobacterales_Representative/phyloP_CONACC/*.wig')[:]:
for infile_name in glob.glob('../Results/Order_Enterobacterales_Mixed/phyloP_CONACC/*.wig')[:]:
    with open(infile_name, 'r') as infile:
        tempy = infile.readlines()
        data = [float(i) for i in tempy[1:]]
        dicty_phylop[infile_name.split('/')[-1].strip('.wig')] = data
print(len(dicty_phylop.keys()))

In [ ]:
# dicty_hyphy = {}
# dicty_w = {}
# for infile_name in glob.glob('../Results/Order_Enterobacterales_Mixed/Hyphy_JC/*.rates')[:]:
#     with open(infile_name, 'r') as infile:
#         tempy = infile.readlines()
#         dicty_w[infile_name.split('/')[-1].strip('.rates')] = float(tempy[1].split(',')[2])
#         rates = [float(i.split(',')[1]) for i in tempy[1:]]
#         meany = np.mean(rates)
#         dicty_hyphy[infile_name.split('/')[-1].strip('.rates')] = [i/meany for i in rates]
# print(len(dicty_hyphy.keys()))

In [ ]:
dicty_hyphy = {}

for infile_name in glob.glob('../Results/Order_Enterobacterales_Mixed/Hyphy_GTR/*.site-rates.json')[:]:
    gene_name = infile_name.split('/')[-1].strip('.site-rates.json')
    with open(infile_name) as infile:
        site_rate_data = json.load(infile)
    positions = [int(i) for i in site_rate_data['Relative site rate estimates'].keys()]
    rates = []
    for position in range(min(positions), max(positions)+1):
        rates.append(site_rate_data['Relative site rate estimates'][str(position)]['MLE'])
    meany = np.mean(rates)
    dicty_hyphy[gene_name] = [i/meany for i in rates]


print(len(dicty_hyphy.keys()))


## Trim 5' terminal gaps according to the E. coli reference sequence
Should this be done before running the models???


In [ ]:
print(np.mean([len(i) for i in seq_dicty[base_genome].values()]))
print(np.mean([len(i) for i in dicty_phylop.values()]))
print(np.mean([len(i) for i in dicty_hyphy.values()]))

for i, j in seq_dicty[base_genome].items():
    if j[0:3] == '---':
        first_bases = []
        for base in ['A', 'T', 'G', 'C']:
            first_bases.append(j.find(base))
        first_base = min(first_bases)
        
        seq_dicty[base_genome][i] = j[first_base:]
        
        if i in dicty_phylop.keys():
            dicty_phylop[i] = dicty_phylop[i][first_base:]
        
        if i in dicty_hyphy.keys():
            dicty_hyphy[i] = dicty_hyphy[i][first_base:]

        if i in conservation_dicty.keys():
            conservation_dicty[i] = conservation_dicty[i][int(first_base/3):]
                
        if i in nt_consensus_dicty.keys():
            nt_consensus_dicty[i] = nt_consensus_dicty[i][first_base:]
            
print(np.mean([len(i) for i in seq_dicty[base_genome].values()]))
print(np.mean([len(i) for i in dicty_phylop.values()]))
print(np.mean([len(i) for i in dicty_hyphy.values()]))

## And 3' terminal gaps according to the E. coli reference sequence

In [ ]:
print(np.mean([len(i) for i in seq_dicty[base_genome].values()]))
print(np.mean([len(i) for i in dicty_phylop.values()]))
print(np.mean([len(i) for i in dicty_hyphy.values()]))


for i, j in seq_dicty[base_genome].items():
    if j[-3:] == '---':
        last_bases = []
        for base in ['A', 'T', 'G', 'C']:
            last_bases.append(j[::-1].find(base))
        last_base = min(last_bases)
        seq_dicty[base_genome][i] = j[:len(j)-last_base]
        if i in dicty_phylop.keys():
            dicty_phylop[i] = dicty_phylop[i][:len(j)-last_base]
            
        if i in dicty_hyphy.keys():
            dicty_hyphy[i] = dicty_hyphy[i][:len(j)-last_base]

        if i in conservation_dicty.keys():
            conservation_dicty[i] = conservation_dicty[i][:int((len(j)-last_base)/3)]

        if i in nt_consensus_dicty.keys():
            nt_consensus_dicty[i] = nt_consensus_dicty[i][:len(j)-last_base]
print(np.mean([len(i) for i in seq_dicty[base_genome].values()]))
print(np.mean([len(i) for i in dicty_phylop.values()]))
print(np.mean([len(i) for i in dicty_hyphy.values()]))

In [ ]:
#####Consensus terminals?"
# print(np.mean([len(i) for i in seq_dicty[base_genome].values()]))
# print(np.mean([len(i) for i in dicty_phylop.values()]))

# for genome in seq_dicty.keys():
#     longest = []
#     for i, j in seq_dicty[genome].items():
#         if j[0:3] == '---':
#             first_bases = []
#             for base in ['A', 'T', 'G', 'C']:
#                 first_bases.append(j.find(base))
#             first_base = min(first_bases)
#             longest.append(first_base)
#     strict_first = max(longest)
#     seq_dicty[base_genome][i] = j[strict_first:]
#     dicty_phylop[i] = dicty_phylop[i][strict_first:]
    
# print(np.mean([len(i) for i in seq_dicty[base_genome].values()]))
# print(np.mean([len(i) for i in dicty_phylop.values()]))

# Investigate relationships between the metrics on the full dataset

In [ ]:
phylop = []
for key, val in dicty_phylop.items():
    phylop.extend(dicty_phylop[key])   
fig, ax = plt.subplots(figsize=(6,4))
ax.hist(phylop, 20, color='k', alpha=0.5)
ax.set_xlabel('phyloP score')
ax.set_ylabel('counts')

In [ ]:
hyphy = []
for key, val in dicty_hyphy.items():
    hyphy.extend(dicty_hyphy[key])   
fig, ax = plt.subplots(figsize=(6,4))
ax.hist(hyphy, 20, color='k', alpha=0.5)
ax.set_xlabel('hyphy (rel subs rate)')
ax.set_yscale('log')
ax.set_ylabel('counts')

# Repeat above averaging over genes

In [ ]:
# phylop = []
# for key, val in dicty_phylop.items():
#     phylop.append(np.mean(dicty_phylop[key])) 
# fig, ax = plt.subplots(figsize=(6,4))
# ax.hist(phylop, 20, color='gray')
# ax.set_xlabel('mean phyloP score per gene')
# ax.set_ylabel('counts')
# plt.tight_layout()

In [ ]:
# w = []
# for key, val in dicty_w.items():
#     w.append(dicty_w[key])
# fig, ax = plt.subplots(figsize=(6,4))
# ax.hist(w, 20, color='gray')
# ax.set_xlabel('hyphy (w)')
# ax.set_ylabel('counts')
# plt.tight_layout()

In [ ]:
# variable_x = []
# variable_y = []
# for key in seq_dicty[base_genome].keys():
#     if key in dicty_phylop.keys() and key in dicty_hyphy.keys():
#         variable_x.extend(dicty_hyphy[key])
#         variable_y.extend(dicty_phylop[key])
# #         mean_val = np.mean(dicty_phylop[key])
# #         variable_y.extend([i/mean_val for i in dicty_phylop[key]])

# print(stats.spearmanr(variable_x, variable_y))

# # plt.figure()
# # plt.semilogx(variable_x, variable_y, 'bo', alpha=0.2)

In [ ]:
# variable_x = []
# variable_y = []
# for key in seq_dicty[base_genome].keys():
#     if key in dicty_phylop.keys() and key in dicty_hyphy_NOT_NORMALIZED.keys():
#         variable_x.append(np.mean(dicty_phylop[key]))
#         variable_y.append(np.mean(dicty_hyphy_NOT_NORMALIZED[key]))
# print(len(variable_x), len(variable_y))
# print(stats.spearmanr(variable_x, variable_y))

# Look at length and position-dependent scores

In [ ]:
ivals = []
plot_vals_phylop = []
plot_vals_hyphy = []

for i in range(150):
    tempy = []
    for gene_name, vals in dicty_phylop.items():
        if len(vals) > 150:
#             if conservation_dicty[gene_name][int(np.floor(i/3))] == 'C':
                tempy.extend([vals[i]])
    plot_vals_phylop.append(np.median(tempy))
    
    tempy = []
    for gene_name, vals in dicty_hyphy.items():
        if len(vals) > 150:
#             if conservation_dicty[gene_name][int(np.floor(i/3))] == 'C':
                tempy.extend([vals[i]])
    plot_vals_hyphy.append(np.median(tempy))
    
    ivals.append(i)

In [ ]:
ivals_r = []
plot_vals_phylop_r = []
plot_vals_hyphy_r = []

for i in range(150, 0, -1):
    tempy = []
    for gene_name, vals in dicty_phylop.items():
        if len(vals) > 150:
#             if conservation_dicty[gene_name][int(np.floor(i/3))] == 'C':
                tempy.extend([vals[-i]])
    plot_vals_phylop_r.append(np.median(tempy))
    
    tempy = []
    for gene_name, vals in dicty_hyphy.items():
        if len(vals) > 150:
#             if conservation_dicty[gene_name][int(np.floor(i/3))] == 'C':
                tempy.extend([vals[-i]])
    plot_vals_hyphy_r.append(np.median(tempy))
    
    ivals_r.append(-i)

In [ ]:
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(221)
ax1.plot(ivals, plot_vals_phylop)
ax1.set_ylabel('phyloP')
ax2 = fig.add_subplot(222, sharey=ax1)
ax2.plot(ivals_r, plot_vals_phylop_r)
ax3 = fig.add_subplot(223, sharex=ax1)
ax3.plot(ivals, plot_vals_hyphy)
ax3.set_ylabel('hyphy')
ax3.set_xlabel('NT position')
ax4 = fig.add_subplot(224, sharey=ax3, sharex=ax2)
ax4.plot(ivals_r, plot_vals_hyphy_r)
ax4.set_xlabel('NT position')
plt.setp(ax1.get_xticklabels(), visible=False);
plt.setp(ax2.get_xticklabels(), visible=False);
plt.setp(ax2.get_yticklabels(), visible=False);
plt.setp(ax4.get_yticklabels(), visible=False);

In [ ]:
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(221)
ax1.plot(ivals[0::3], plot_vals_phylop[0::3], 'bo')
ax1.plot(ivals[1::3], plot_vals_phylop[1::3], 'go')
ax1.plot(ivals[2::3], plot_vals_phylop[2::3], 'ro')
ax1.set_ylabel('phyloP')
ax2 = fig.add_subplot(222, sharey=ax1)
ax2.plot(ivals_r[0::3], plot_vals_phylop_r[0::3], 'bo')
ax2.plot(ivals_r[1::3], plot_vals_phylop_r[1::3], 'go')
ax2.plot(ivals_r[2::3], plot_vals_phylop_r[2::3], 'ro')
ax3 = fig.add_subplot(223, sharex=ax1)
ax3.plot(ivals[0::3], plot_vals_hyphy[0::3], 'bo')
ax3.plot(ivals[1::3], plot_vals_hyphy[1::3], 'go')
ax3.plot(ivals[2::3], plot_vals_hyphy[2::3], 'ro')
ax3.set_ylabel('hyphy')
ax3.set_xlabel('NT position')
ax4 = fig.add_subplot(224, sharey=ax3, sharex=ax2)
ax4.plot(ivals_r[0::3], plot_vals_hyphy_r[0::3], 'bo')
ax4.plot(ivals_r[1::3], plot_vals_hyphy_r[1::3], 'go')
ax4.plot(ivals_r[2::3], plot_vals_hyphy_r[2::3], 'ro')
ax4.set_xlabel('NT position')
plt.setp(ax1.get_xticklabels(), visible=False);
plt.setp(ax2.get_xticklabels(), visible=False);
plt.setp(ax2.get_yticklabels(), visible=False);
plt.setp(ax4.get_yticklabels(), visible=False);

# Compare 2-fold and 4-fold redundant sites, conserved and not

In [ ]:
twofold_synon_phylop = []
fourfold_synon_phylop = []

twofold_synon_hyphy = []
fourfold_synon_hyphy = []
for i,j in seq_dicty[base_genome].items():
    if i not in dicty_hyphy.keys():
        continue
    if i not in dicty_phylop.keys():
        continue 
    if len(j) < 150:
        continue
    j_codons = [j[nt:nt+3] for nt in range(0, len(j), 3)]
    for pos, codon in enumerate(j_codons):
        if pos < 100:
            continue
        if codon == '---':
            continue
        if conservation_dicty[i][pos] != 'C':
            continue
        if codon in redundant_codons:
            fourfold_synon_phylop.append(dicty_phylop[i][(pos*3)+2])
            fourfold_synon_hyphy.append(dicty_hyphy[i][(pos*3)+2])
        elif codon_to_aa_dict[codon] in ['F', 'H', 'Q', 'N', 'K', 'D', 'E']:
            twofold_synon_phylop.append(dicty_phylop[i][(pos*3)+2])
            twofold_synon_hyphy.append(dicty_hyphy[i][(pos*3)+2])
            
fig = plt.figure(figsize=(12,4))
ax1 = fig.add_subplot(121)
ax1.boxplot([twofold_synon_phylop, fourfold_synon_phylop]);
ax1.set_ylabel('phyloP score')
ax1.set_xticklabels(['2-fold\nredundant', '4-fold\nredundant'], rotation=0)

ax2 = fig.add_subplot(122)
ax2.boxplot([twofold_synon_hyphy, fourfold_synon_hyphy]);
ax2.set_ylabel('hyphy score')
ax2.set_xticklabels(['2-fold\nredundant', '4-fold\nredundant'], rotation=0)
plt.tight_layout()

print('##### phyloP analysis')
print(len(twofold_synon_phylop), len(fourfold_synon_phylop))
print(np.mean(twofold_synon_phylop), np.mean(fourfold_synon_phylop))
print(stats.ranksums(twofold_synon_phylop, fourfold_synon_phylop))

print('##### hyphy analysis')
print(len(twofold_synon_hyphy), len(fourfold_synon_hyphy))
print(np.mean(twofold_synon_hyphy), np.mean(fourfold_synon_hyphy))
print(stats.ranksums(twofold_synon_hyphy, fourfold_synon_hyphy))

In [ ]:
fourfold_synon_cons_phylop = []
fourfold_synon_var_phylop = []
fourfold_synon_cons_hyphy = []
fourfold_synon_var_hyphy = []

for i,j in seq_dicty[base_genome].items():
    if i not in dicty_hyphy.keys():
        continue
    if i not in dicty_phylop.keys():
        continue
    if len(j) < 150:
        continue
    j_codons = [j[nt:nt+3] for nt in range(0, len(j), 3)]
    for pos, codon in enumerate(j_codons):
        if pos < 50:
            continue
        if codon == '---':
            continue
        if conservation_dicty[i][pos] == 'C':
            if codon in redundant_codons:
                fourfold_synon_cons_phylop.append(dicty_phylop[i][(pos*3)+2])
                fourfold_synon_cons_hyphy.append(dicty_hyphy[i][(pos*3)+2])
        elif conservation_dicty[i][pos] == 'V':
            if codon in redundant_codons:
                fourfold_synon_var_phylop.append(dicty_phylop[i][(pos*3)+2])
                fourfold_synon_var_hyphy.append(dicty_hyphy[i][(pos*3)+2])
    
fig = plt.figure(figsize=(12,4))
ax1 = fig.add_subplot(121)
ax1.boxplot([fourfold_synon_cons_phylop, fourfold_synon_var_phylop]);
ax1.set_ylabel('phyloP score')
ax1.set_xticklabels(['Fully conserved aa', 'Variable aa'], rotation=0)

ax2 = fig.add_subplot(122)
ax2.boxplot([fourfold_synon_cons_hyphy, fourfold_synon_var_hyphy]);
ax2.set_ylabel('hyphy score')
ax2.set_xticklabels(['Fully conserved aa', 'Variable aa'], rotation=0)
plt.tight_layout()

print('##### phyloP analysis')
print(len(fourfold_synon_cons_phylop), len(fourfold_synon_var_phylop))
print("Category Means:", np.mean(fourfold_synon_cons_phylop), np.mean(fourfold_synon_var_phylop))
print("Category Medians:", np.median(fourfold_synon_cons_phylop), np.median(fourfold_synon_var_phylop))

print(stats.ranksums(fourfold_synon_cons_phylop, fourfold_synon_var_phylop))

print('##### hyphy analysis')
print(len(fourfold_synon_cons_hyphy), len(fourfold_synon_var_hyphy))
print("Category Means:", np.mean(fourfold_synon_cons_hyphy), np.mean(fourfold_synon_var_hyphy))
print("Category Medians:", np.median(fourfold_synon_cons_hyphy), np.median(fourfold_synon_var_hyphy))
print(stats.ranksums(fourfold_synon_cons_hyphy, fourfold_synon_var_hyphy))

# Defining a SD threshold strength

In [ ]:
with open('../Data/json-energyRef-CCUCCU.txt') as infile:
    fragment_energy_dict = json.load(infile)
asd_seq = 'CCTCCT'

In [ ]:
real_utrs = []
include = 20
genome = list(SeqIO.parse('/Users/adamhockenberry/Projects/2017/Genome_database/Data/GBFs/511145.12.PATRIC.gbf', 'genbank'))
for record in genome:
    for feature in record.features[:]:
        if feature.type == 'CDS':
            if feature.location.strand == 1:
                real_utrs.append(record.seq[feature.location.start-include:feature.location.start])
            elif feature.location.strand == -1:
                real_utrs.append(record.seq[feature.location.end:feature.location.end+include].reverse_complement())

In [ ]:
randomizations = 1
real_utrs = [str(i) for i in real_utrs]
strongest_real = []
strongest_fake = []
strongest_fake_dict = {}
for randomization in range(randomizations):
    strongest_fake_dict[randomization] = []


for utr in real_utrs[:]:
    energy_list = []
    for i in range(0, len(utr)-len(asd_seq)-4):
        fragment = utr[i:i+len(asd_seq)]
        energy = fragment_energy_dict[fragment.replace('T', 'U')]
        energy_list.append(energy)
    strongest_real.append(min(energy_list))
    
    for randomization in range(randomizations):
        energy_list = []
        utr_rand = np.array(list(utr))
        np.random.shuffle(utr_rand)
        utr_rand = ''.join(list(utr_rand))
        for i in range(0, len(utr_rand)-len(asd_seq)-4):
            fragment = utr_rand[i:i+len(asd_seq)]
            energy = fragment_energy_dict[fragment.replace('T', 'U')]
            energy_list.append(energy)
        strongest_fake.append(min(energy_list))
        strongest_fake_dict[randomization].append(min(energy_list))
print(len(strongest_real))
print(len(strongest_fake))
        

In [ ]:
plt.figure()
plt.hist(strongest_fake, 60, normed=True, alpha=0.5, color='gray')
plt.hist(strongest_real, 60, normed=True, alpha=0.5)
plt.axvline(-4.0, c='r')
# plt.axvline(-3.75, c='r', alpha=0.5)
plt.axvline(-5.5, c='r', alpha=0.5)

In [ ]:
testy = []
for threshold in np.arange(0, -10, -0.1):
    print(threshold)
    actual = len([i for i in strongest_real if i < threshold])
    randos = [len([i for i in k if i < threshold]) for j,k in strongest_fake_dict.items()]
    print(actual, np.mean(randos))
    testy.append((actual-np.mean(randos)))
#     testy.append((actual-np.mean(randos))/np.std(randos))
    if actual > max(randos):
        print('hereeeeeeeee')

In [ ]:
plt.figure()
plt.plot(np.arange(0, -10, -0.1), testy)
plt.axvline(-4.5)
plt.axvline(-4.0)
plt.axvline(-3.5)

# Actually dig into data to test for conservation of individual X-mers
Still trying to decide on the best strategy. My current thinking is to look ONLY at 4-fold redundant codon sites that are within a strong-ish aSD binding site and for each one compare the conservation at that site to the same synonymous codon somewhere else in the same gene. With a list of these, can compare whether there is any conservation difference

## Positive control data. Find genes that have another gene directly 3' of them which are thus likely to contain a real SD sequence in their 3' terminus

In [ ]:
preceding_genes = []
starts_positive_strand = []
starts_negative_strand = []
genome = list(SeqIO.parse('/Users/adamhockenberry/Projects/2017/Genome_database/Data/GBFs/511145.12.PATRIC.gbf', 'genbank'))
for record in genome:
    for feature in record.features:
        if feature.type == 'CDS':
            if feature.location.strand == 1:
                starts_positive_strand.append(feature.location.start)
            elif feature.location.strand == -1:
                starts_negative_strand.append(feature.location.end)
for i,j in enumerate(starts_positive_strand):
    starts_positive_strand[i] = list(range(j-20,j+10,1))
for i,j in enumerate(starts_negative_strand):
    starts_negative_strand[i] = list(range(j-10,j+20,1))
for record in genome:
    for feature in record.features:
        if feature.type == 'CDS':
            if feature.location.strand == 1:
                end = feature.location.end
                for i in starts_positive_strand:
                    if end in i:
                        preceding_genes.append(feature.qualifiers['locus_tag'][0])
            elif feature.location.strand == -1:
                end = feature.location.start
                for i in starts_negative_strand:
                    if end in i:
                        preceding_genes.append(feature.qualifiers['locus_tag'][0])
                        
preceding_genes = [i for i in seq_dicty[base_genome].keys() if i in preceding_genes]
not_preceding_genes = [i for i in seq_dicty[base_genome].keys() if i not in preceding_genes]

In [ ]:
len(preceding_genes), len(not_preceding_genes), len(seq_dicty[base_genome].keys())

## Comparing conservation of a particular 4-fold redundant position that is within a strong binding sequence to a randomly chosen identical 4-fold redundant codon elsewhere in the sequence
Note I'm currently looking at a 6nt aSD sequence and only considering 4-fold redundant codons in positions 2,3,4, or 5. I'm also taking care not to double count any one position

In [ ]:
def get_sd_sites(nt_sequence, asd_seq, threshold, fragment_energy_dict, upper_threshold=-20, min_position=100, max_position=-50, three_prime_analysis=False):
    codon_list = [nt_sequence[nt:nt+3] for nt in range(0, len(nt_sequence), 3)]
    energy_list = []
    for i in range(0, len(nt_sequence)-len(asd_seq)):
        fragment = nt_sequence[i:i+len(asd_seq)]
        if fragment.count('-') == 0:
            energy = fragment_energy_dict[fragment.replace('T', 'U')]
            energy_list.append(energy)
        else:
            energy_list.append(np.nan)
    #######This gets all positions below the given threshold
    sd_positions_all_nts = [list(range(i, i+len(asd_seq))) for i,j in enumerate(energy_list) if j < threshold]
    sd_positions_all_nts = [item for sublist in sd_positions_all_nts for item in sublist]
    sd_positions_all_nts = list(set(sd_positions_all_nts))
    
    #######This gets all "start" position within the threshold range. Subtle but important difference
    if three_prime_analysis:
        sd_positions_start = np.array([i for i,j in enumerate(energy_list) if upper_threshold < j and j < threshold and i > len(nt_sequence)-50])
    else:
        sd_positions_start = np.array([i for i,j in enumerate(energy_list) if upper_threshold < j and j < threshold and i > min_position and i < len(nt_sequence)+max_position])
    
    np.random.shuffle(sd_positions_start) #So I'm not always analyzing the 5' ones every single time
    return sd_positions_all_nts, sd_positions_start, energy_list, codon_list

def get_control_sites_codon(nt_sequence, codon_list, sd_positions_all_nts, analyzed_positions, codon, min_position=100, max_position=-50):
    control_sites = [site*3+2 for site, x in enumerate(codon_list) if x == codon]
    control_sites = [site for site in control_sites if site > min_position and site < len(nt_sequence)+max_position]
    control_sites = [site for site in control_sites if site not in sd_positions_all_nts]
    control_sites = [site for site in control_sites if site not in analyzed_positions]
    return control_sites

def get_control_sites_tri_nt(nt_sequence, codon_list, sd_positions_all_nts, analyzed_positions, tri_nt, redundant_codons, min_position=100, max_position=-50):
    control_sites = [m.start()+1 for m in re.finditer(tri_nt, nt_sequence)]
    control_sites = [site for site in control_sites if site > min_position and site < len(nt_sequence)+max_position]
    control_sites = [site for site in control_sites if site%3 == 2]
    control_sites = [site for site in control_sites if nt_sequence[site-2:site+1] in redundant_codons]
    control_sites = [site for site in control_sites if site not in sd_positions_all_nts]
    control_sites = [site for site in control_sites if site not in analyzed_positions]
    return control_sites

def get_possible_energies(fragment, codon, all_codons, modifier, fragment_energy_dict):
    possible_energies = []
    for possible_codon in all_codons:
        if possible_codon != codon:
            if modifier == 2:
                possible_fragment = possible_codon + fragment[-3:]
            elif modifier == 3:
                possible_fragment = fragment[0] + possible_codon + fragment[-2:]
            elif modifier == 1:
                possible_fragment = possible_codon[-2:] + fragment[-4:]
            elif modifier == 4:
                possible_fragment = fragment[0:2] + possible_codon + fragment[-1]
            assert len(possible_fragment) == 6
            possible_energies.append(fragment_energy_dict[possible_fragment.replace('T', 'U')])
    return possible_energies


def paired_nt_analysis(dicty_to_analyze,\
                      genome_seq_dicty,\
                      fragment_energy_dict,\
                      asd_seq,\
                      conservation_dicty,\
                      redundant_codons=redundant_codons,\
                      threshold=-4.5,\
                      upper_threshold=-20,\
                      min_position=100,\
                      max_position=-50,\
                      subset_names=False,\
                      randomization=False,\
                      only_conserved_aa=False,\
                      three_prime_analysis=False,\
                      internal_control='',
                      nt_strength_restriction=''):
    sds = []
    others = []
    for gene_name, seq in list(genome_seq_dicty.items())[:]:

        analyzed_positions = []

        if gene_name not in dicty_to_analyze.keys():
            continue
        if subset_names:
            if gene_name not in subset_names:
                continue
        ##################################################################################################
        ###GET LOCATIONS OF ALL SD-LIKE SEQUENCES THAT I MIGHT WANT TO ANALYZE        
        sd_positions_all_nts, sd_positions_start, energy_list, codon_list =\
                        get_sd_sites(seq, asd_seq, threshold, fragment_energy_dict,\
                                     upper_threshold=upper_threshold, min_position=min_position,\
                                     max_position=max_position, three_prime_analysis=three_prime_analysis)
        
        for i in sd_positions_start:
            energy = energy_list[i]
            modifiers = []
            if i%3 == 0:
                modifiers = [2]
            elif i%3 == 2:
                modifiers = [3]
            elif i%3 == 1:
                modifiers = [1,4]
            for modifier in modifiers:
                if only_conserved_aa == True:
                    if conservation_dicty[gene_name][int(np.floor((i+modifier)/3.))] != 'C':
                        continue
#                     if conservation_dicty[gene_name][int(np.floor((i+modifier+3)/3.))] != 'C':
#                         continue
#                     if conservation_dicty[gene_name][int(np.floor((i+modifier-3)/3.))] != 'C':
#                         continue
                codon = seq[i+modifier-2:i+modifier+1]
                tri_nt = seq[i+modifier-1:i+modifier+2]
                if codon in redundant_codons:
                    ##################################################################################################
                    ###RUN A FILTER ACCORDING TO WHETHER I ONLY WANT TO SELECT STRONGEST OR WEAKEST POSSIBLE SEQUENCES
                    if nt_strength_restriction == '':
                        pass
                    else:
                        fragment = genome_seq_dicty[gene_name][i:i+len(asd_seq)]
                        all_codons = aa_to_codon_dict[codon_to_aa_dict[codon]]
                        possible_energies = get_possible_energies(fragment, codon, all_codons, modifier, fragment_energy_dict)
                        if nt_strength_restriction == 'CURRENT_IS_STRONGEST':
                            if energy > min(possible_energies):
                                continue
                        elif nt_strength_restriction=='CURRENT_IS_WEAKEST':
                            if energy < max(possible_energies):
                                continue
                        else:
                            print('YOU DID NOT PROPERLY SPECIFY THE NT_STRENGTH_RESTRICTION VARIABLE')
                            break
                    ##################################################################################################
                    ###GET CONTROL SITES ACCORDING TO EITHER OF TWO DIFFERENT NULL MODELS
                    if internal_control == 'CODON':
                        control_sites =\
                            get_control_sites_codon(seq, codon_list, sd_positions_all_nts, analyzed_positions, codon,\
                                                   min_position=min_position, max_position=max_position)
                    elif internal_control == 'TRINT':
                        control_sites =\
                            get_control_sites_tri_nt(seq, codon_list, sd_positions_all_nts, analyzed_positions, tri_nt, redundant_codons,\
                                                    min_position=min_position, max_position=max_position)
                    else:
                        print("YOU DID NOT PROPERLY SPECIFY A CONTROL METHOD")
                        break
                    ##################################################################################################
                    ###DECIDE WHETHER TO LIMIT ACCORDING TO CONSERVATION OF THE AMINO ACID 
                    if only_conserved_aa == True:
                        control_sites = [site for site in control_sites if conservation_dicty[gene_name][int(np.floor(site/3.))] == 'C']
#                         control_sites = [site for site in control_sites if conservation_dicty[gene_name][int(np.floor((site+3)/3.))] == 'C']
#                         control_sites = [site for site in control_sites if conservation_dicty[gene_name][int(np.floor((site-3)/3.))] == 'C']

                    ##################################################################################################
                    ###IF ALL LOOKS WELL CHOOSE A CONTROL AND GO FORWARD WITH ANALYSIS
                    if len(control_sites) > 0 and i+modifier not in analyzed_positions:
                        choice = random.choice(control_sites)
                        if randomization==True:
                            if np.random.choice(['H', 'T']) == 'H': #Heads or tails coinflip for random assignment. Smiley face.
                                sds.append(dicty_to_analyze[gene_name][i+modifier])
                                others.append(dicty_to_analyze[gene_name][choice])
                            else:
                                sds.append(dicty_to_analyze[gene_name][choice])
                                others.append(dicty_to_analyze[gene_name][i+modifier])
                        else:
                            sds.append(dicty_to_analyze[gene_name][i+modifier])
                            others.append(dicty_to_analyze[gene_name][choice])
                                                    
                        analyzed_positions.append(i+modifier)
                        analyzed_positions.append(choice)
                        #######################################################################
                        ###Fail safe to make sure I chose the right codons and context controls
                        if internal_control == 'CODON':
                            assert seq_dicty[base_genome][gene_name][i+modifier-2:i+modifier+1] == seq_dicty[base_genome][gene_name][choice-2:choice+1]
                        if internal_control == 'TRINT':
                            assert seq_dicty[base_genome][gene_name][i+modifier-1:i+modifier+2] == seq_dicty[base_genome][gene_name][choice-1:choice+2]

    return sds, others

In [ ]:
def all_nt_analysis(dicty_to_analyze,\
                      genome_seq_dicty,\
                      fragment_energy_dict,\
                      asd_seq,\
                      conservation_dicty,\
                      redundant_codons=redundant_codons,\
                      threshold=-4.5,\
                      subset_names=False,\
                      randomization=False,\
                      only_conserved_aa=False,\
                      three_prime_analysis=False):
    sds = []
    control_codon = []
    control_tri_nt = []
    
    for gene_name, seq in list(genome_seq_dicty.items())[:]:
        analyzed_positions = []


        if gene_name not in dicty_to_analyze.keys():
            continue
        if subset_names:
            if gene_name not in subset_names:
                continue


                
        sd_positions_all_nts, sd_positions_start, energy_list, codon_list =\
                        get_sd_sites(seq, asd_seq, threshold, fragment_energy_dict, min_position=100, max_position=-50, three_prime_analysis=three_prime_analysis)
        possible_codons = []
        possible_tri_nts = []
        for i in sd_positions_start:
            energy = energy_list[i]
            modifiers = []
            if i%3 == 0:
                modifiers = [2]
            elif i%3 == 2:
                modifiers = [3]
            elif i%3 == 1:
                modifiers = [1,4]
            for modifier in modifiers:
                if only_conserved_aa == True:
                    if conservation_dicty[gene_name][int(np.floor((i+modifier)/3.))] != 'C':
                        continue
#                     if conservation_dicty[gene_name][int(np.floor((i+modifier+3)/3.))] != 'C':
#                         continue
#                     if conservation_dicty[gene_name][int(np.floor((i+modifier-3)/3.))] != 'C':
#                         continue
                codon = seq[i+modifier-2:i+modifier+1]
                tri_nt = seq[i+modifier-1:i+modifier+2]
                if codon in redundant_codons and i+modifier not in analyzed_positions:
                    sds.append(dicty_to_analyze[gene_name][i+modifier])
                    possible_codons.append(codon)
                    possible_tri_nts.append(tri_nt)
                    analyzed_positions.append(i+modifier)
    
        possible_codons = list(set(possible_codons))
        for codon in possible_codons:
            locs = [site*3+2 for site, x in enumerate(codon_list) if x == codon]
            locs = [site for site in locs if site not in analyzed_positions and site > 100 and site < len(seq)-50]
            if only_conserved_aa == True:
                locs = [site for site in locs if conservation_dicty[gene_name][int(np.floor(site/3.))] == 'C']
#                 locs = [site for site in locs if conservation_dicty[gene_name][int(np.floor((site+3)/3.))] == 'C']
#                 locs = [site for site in locs if conservation_dicty[gene_name][int(np.floor((site-3)/3.))] == 'C']
            control_codon.extend([dicty_to_analyze[gene_name][site] for site in locs])
        
        possible_tri_nts = list(set(possible_tri_nts))
        for tri_nt in possible_tri_nts:
            locs = [m.start()+1 for m in re.finditer(tri_nt, seq)]
            locs = [site for site in locs if site > 100 and site < len(seq)-50]
            locs = [site for site in locs if site%3 == 2]
            locs = [site for site in locs if seq[site-2:site+1] in redundant_codons]
            if only_conserved_aa == True:
                locs = [site for site in locs if conservation_dicty[gene_name][int(np.floor(site/3.))] == 'C']
#                 locs = [site for site in locs if conservation_dicty[gene_name][int(np.floor((site+3)/3.))] == 'C']
#                 locs = [site for site in locs if conservation_dicty[gene_name][int(np.floor((site-3)/3.))] == 'C']

            control_tri_nt.extend([dicty_to_analyze[gene_name][site] for site in locs])
        
    return sds, control_codon, control_tri_nt

In [ ]:
dicty_to_analyze = dicty_hyphy
threshold=-4.5
upper_threshold=-20
subset_names=False
only_conserved_aa=False
three_prime_analysis=False
controls=('CODON', 'TRINT')
nt_strength_restriction=''
# nt_strength_restriction='CURRENT_IS_STRONGEST'

with open('../Data/json-energyRef-CCUCCU.txt') as infile:
    fragment_energy_dict = json.load(infile)
asd_seq = 'CCUCCU'


file_name_modifiers = '{}_threshold={}_Conserved={}_ThreePrime={}'.format('hyphy', threshold, only_conserved_aa, three_prime_analysis)


actual_ratios_codon = []
shuffled_ratios_codon = []
actual_ratios_trint = []
shuffled_ratios_trint = []
for i in range(20):
    print('#####{}'.format(i))
    sds, others = paired_nt_analysis(dicty_to_analyze,\
                                     seq_dicty[base_genome],\
                                     fragment_energy_dict,\
                                     asd_seq,\
                                     conservation_dicty,\
                                     threshold=threshold,\
                                     upper_threshold=upper_threshold,\
                                     randomization=False,\
                                     subset_names=subset_names,\
                                     only_conserved_aa=only_conserved_aa,\
                                     three_prime_analysis=three_prime_analysis,\
                                     internal_control=controls[0],\
                                     nt_strength_restriction=nt_strength_restriction)
    actual_ratios_codon.append(np.mean(sds)/np.mean(others))
    print('Lengths:{}, {}, Means:{}, {}, Ratio:{}'.format(len(sds), len(others), np.mean(sds), np.mean(others), np.mean(sds)/np.mean(others)))
    sds, others = paired_nt_analysis(dicty_to_analyze,\
                                     seq_dicty[base_genome],\
                                     fragment_energy_dict,\
                                     asd_seq,\
                                     conservation_dicty,\
                                     threshold=threshold,\
                                     upper_threshold=upper_threshold,\
                                     randomization=True,\
                                     subset_names=subset_names,\
                                     only_conserved_aa=only_conserved_aa,\
                                     three_prime_analysis=three_prime_analysis,\
                                     internal_control=controls[0],\
                                     nt_strength_restriction=nt_strength_restriction)
    shuffled_ratios_codon.append(np.mean(sds)/np.mean(others))
    
    sds, others = paired_nt_analysis(dicty_to_analyze,\
                                     seq_dicty[base_genome],\
                                     fragment_energy_dict,\
                                     asd_seq,\
                                     conservation_dicty,\
                                     threshold=threshold,\
                                     upper_threshold=upper_threshold,\
                                     randomization=False,\
                                     subset_names=subset_names,\
                                     only_conserved_aa=only_conserved_aa,\
                                     three_prime_analysis=three_prime_analysis,\
                                     internal_control=controls[1],\
                                     nt_strength_restriction=nt_strength_restriction)
    actual_ratios_trint.append(np.mean(sds)/np.mean(others))
    print('Lengths:{}, {}, Means:{}, {}, Ratio:{}'.format(len(sds), len(others), np.mean(sds), np.mean(others), np.mean(sds)/np.mean(others)))
    sds, others = paired_nt_analysis(dicty_to_analyze,\
                                     seq_dicty[base_genome],\
                                     fragment_energy_dict,\
                                     asd_seq,\
                                     conservation_dicty,\
                                     threshold=threshold,\
                                     upper_threshold=upper_threshold,\
                                     randomization=True,\
                                     subset_names=subset_names,\
                                     only_conserved_aa=only_conserved_aa,\
                                     three_prime_analysis=three_prime_analysis,\
                                     internal_control=controls[1],\
                                     nt_strength_restriction=nt_strength_restriction)
    shuffled_ratios_trint.append(np.mean(sds)/np.mean(others))

In [ ]:
dfA = pd.DataFrame({'Actual_rats_codon':actual_ratios_codon})
dfB = pd.DataFrame({'Shuffled_rats_codon':shuffled_ratios_codon})
dfC = pd.DataFrame({'Actual_rats_trint':actual_ratios_trint})
dfD = pd.DataFrame({'Shuffled_rats_trint':shuffled_ratios_trint})
df = pd.concat([dfA[['Actual_rats_codon']],dfB[['Shuffled_rats_codon']],\
               dfC[['Actual_rats_trint']],dfD[['Shuffled_rats_trint']]], axis=1)
df.to_csv('../Results/temp_df_{}_ratios.csv'.format(file_name_modifiers), index=False)

# Unpaired analysis (for visualization)

In [ ]:
all_sds, all_others_codon, all_others_tri_nt = all_nt_analysis(dicty_to_analyze,\
                                 seq_dicty[base_genome],\
                                 fragment_energy_dict,\
                                 asd_seq,\
                                 conservation_dicty,\
                                 threshold=threshold,\
                                 randomization=False,\
                                 subset_names=subset_names,\
                                 only_conserved_aa=only_conserved_aa,\
                                 three_prime_analysis=three_prime_analysis)

In [ ]:
print(len(all_sds), len(all_others_codon), len(all_others_tri_nt))


dfA = pd.DataFrame({'SD-like':all_sds})
dfB = pd.DataFrame({'Control\n(codon)':all_others_codon})
dfC = pd.DataFrame({'Control\n(context)':all_others_tri_nt})

df = pd.concat([dfA[['SD-like']],dfB[['Control\n(codon)']], dfC[['Control\n(context)']]], axis=1)
df.to_csv('../Results/temp_df_{}.csv'.format(file_name_modifiers), index=False)

# Protein Domain analysis

In [ ]:
df = pd.read_csv('../Data/foldingFromPNAS_v2.csv')
df['bNumber'] = ''
df['patricNumber'] = ''
print(len(df.index))

for index in df.index:
    names = df.loc[index]['Gene names'].split(' ')
    for name in names:
        if name in reference_name_dict.keys():
            df.set_value(index, 'bNumber', name)
            try:
                df.set_value(index, 'patricNumber', reference_name_dict[name])
            except KeyError:
                pass

df = df[df['patricNumber'] != '']
print(len(df.index))

### Choose whether to stratify the data according to Lm
...9 billion dof here

In [ ]:
df = df[df['Lm (residues)'].isnull() == False]
# df = df[(df['Lm (residues)'] > 0) & (df['Lm (residues)'] <= 50)]

### Anywayyy, domain_dict will specify what regions in what proteins I want to actually consider 

In [ ]:
df['Domain_beg_nt'] = int()
df['Domain_end_nt'] = int()
for index in df.index:
    domain_beg, domain_end = df.loc[index]['Codon range of domain'].split('-')
    df.set_value(index, 'Domain_beg_nt', int(domain_beg)*3)
    df.set_value(index, 'Domain_end_nt', int(domain_end)*3)

    
df['Domain_beg_in_patric'] = int()
df['Domain_end_in_patric'] = int()
for index in df.index:    
    ref_seq_beg = ref_genome_seq_ref_dict[df.loc[index]['bNumber']][int(df.loc[index]['Domain_beg_nt']):int(df.loc[index]['Domain_beg_nt'])+100]
    ref_seq_end = ref_genome_seq_ref_dict[df.loc[index]['bNumber']][int(df.loc[index]['Domain_end_nt'])-100:int(df.loc[index]['Domain_end_nt'])]
    df.set_value(index, 'Domain_beg_in_patric', ref_genome_seq_patric_dict[df.loc[index]['patricNumber']].find(ref_seq_beg))
    df.set_value(index, 'Domain_end_in_patric', ref_genome_seq_patric_dict[df.loc[index]['patricNumber']].find(ref_seq_end) + 100)


domain_dicts = {}
for index in df.index:
    try:
        domain_dicts[df.loc[index]['patricNumber']].append(df.loc[index]['Domain_end_in_patric'])
    except:
        domain_dicts[df.loc[index]['patricNumber']] = [df.loc[index]['Domain_end_in_patric']]
        
for i,j in domain_dicts.items():
    tempy = []
    for loc in j:
#         tempy.extend(list(range(loc+60, loc+150))) #####IMPORTANT
        tempy.extend(list(range(loc-150, loc))) #####IMPORTANT
    domain_dicts[i] = tempy

In [ ]:
with open('../Data/json-energyRef-CCUCCU.txt') as infile:
    energy_dict = json.load(infile)
asd = 'CCTCCT'
threshold = -4.5

sds = []
others = []
for gene_name, seq in list(seq_dicty[base_genome].items())[:]:
    if gene_name not in domain_dicts:
        continue
    if gene_name not in dicty_hyphy.keys():
        continue
    codon_list = [seq[nt:nt+3] for nt in range(0, len(seq), 3)]
    energy_list = []
    for i in range(0, len(seq)-len(asd)):
        fragment = seq[i:i+len(asd)]
        if fragment.count('-') == 0:
            energy = energy_dict[fragment.replace('T', 'U')]
            energy_list.append(energy)
        else:
            energy_list.append(np.nan)
    analyzed_positions = []
    sd_positions = [list(range(i, i+6)) for i,j in enumerate(energy_list) if j < threshold]
    sd_positions = [item for sublist in sd_positions for item in sublist]
    sd_positions = list(set(sd_positions))
    for i, energy in enumerate(energy_list):
        if i in domain_dicts[gene_name]:
                if energy < threshold and i > 100 and i < len(seq)-50:
#                 if energy < threshold and i > 100 and i < len(seq)-50 and gene_name in expression_subset:
#                 if energy < threshold and i > 50 and i < len(seq)-50 and gene_name in long_genes:
#                 if energy < threshold and i > len(seq)-50 and gene_name in preceding_genes:
#                 if energy < threshold and i > len(seq)-50 and gene_name not in preceding_genes:
                    if i%3 == 0:
                        modifiers = [2]
                    elif i%3 == 2:
                        modifiers = [3]
                    elif i%3 == 1:
                        modifiers = [1,4]
                    for modifier in modifiers:
                        codon = seq[i+modifier-2:i+modifier+1]
                        if codon in redundant_codons:
                            control_sites = [j*3+2 for j, x in enumerate(codon_list) if x == codon]
                            control_sites = [j for j in control_sites if j not in sd_positions and j > 100 and j < len(seq)-50]
                            if len(control_sites) > 0 and i+modifier not in analyzed_positions:
                                sds.append(dicty_hyphy[gene_name][i+modifier])
                                others.append(dicty_hyphy[gene_name][random.choice(control_sites)])
                                analyzed_positions.append(i+modifier)

In [ ]:
print('Length of my data: {} and {}'.format(len(sds),len(others)))
print('Means of each category SD ({}) and non ({})'.format(np.mean(sds), np.mean(others)))
print('Statistical test {}'.format(stats.wilcoxon(sds, others)))

fig, ax = plt.subplots(figsize=(8,4))
ax.hist(np.array(sds)-np.array(others), 40, alpha=0.8);
ax.set_xlabel('Difference\n <---SD less conserved // SD more conserved--->');
ax.set_ylabel('counts');
ax.axvline(0, c='r', linewidth=2)
ax.axvline(np.mean(np.array(sds)-np.array(others)), c='c', linewidth=2)


fig, ax = plt.subplots(figsize=(6,4))
ax.violinplot([np.array(sds), np.array(others)], points=50, widths=0.5, showextrema=False);
ax.set_xticklabels(['', 'SD sites', '', 'Controls'])
ax.set_ylabel('phylop conservation score')

# Re-investigate that whole hypothesis (from my prior initial work) whereby SD sites were (maybe?) enriched after domain ends

## NOTE: this section needs re-done once I've settled more on the code above

In [ ]:
matrix_total = []
# for index in df.index:
# for index in df[df['Lm (residues)'].isnull() == False].index:
# for index in df[(df['Lm (residues)'] > 0)].index:
for index in df[(df['Lm (residues)'] > 5) & (df['Lm (residues)'] < 30)].index:
    domain_beg, domain_end = df.loc[index]['Codon range of domain'].split('-')
    domain_beg = int(domain_beg)*3
    domain_end = int(domain_end)*3
    if domain_end-domain_beg > 105 and domain_end < len(ref_genome_seq_ref_dict[df.loc[index]['bNumber']]) - 100:
        total_seq = str(ref_genome_seq_ref_dict[df.loc[index]['bNumber']][domain_end-100:domain_end+100]).replace('T', 'U')
        matrix_total.append([energy_dict[total_seq[i:i+6]] for i in range(0, len(total_seq)-6)])

In [ ]:
print(np.shape(matrix_total))
plt.figure(figsize=(12,4))
plt.plot(np.mean(matrix_total, axis=0))
plt.axvline(100, c='r', linestyle='--', linewidth=4)

# Testing gene expression differences systematically

### First, tediously get a dictionary linking names in PATRIC annotations to names in RefSeq

In [ ]:
genome_ref = list(SeqIO.parse('/Users/adamhockenberry/Projects/2017/Genome_database/Data/GBFs/511145.12.RefSeq.gbf', 'genbank'))[0]
genome_pat = list(SeqIO.parse('/Users/adamhockenberry/Projects/2017/Genome_database/Data/GBFs/511145.12.PATRIC.gbf', 'genbank'))[0]

locs_dict_ref_positive = {}
locs_dict_ref_negative = {}

locs_dict_pat_positive = {}
locs_dict_pat_negative = {}

ref_genome_seq_ref_dict = {}
ref_genome_seq_patric_dict = {}

for feature_pat in genome_pat.features[:]:
    if feature_pat.type == 'CDS':
        name_pat = feature_pat.qualifiers['locus_tag'][0]
        strand_pat = feature_pat.strand
        if strand_pat == 1:
            locs_dict_pat_positive[name_pat] = list(range(feature_pat.location.start, feature_pat.location.end))
            ref_genome_seq_patric_dict[name_pat] = genome_pat.seq[feature_pat.location.start:feature_pat.location.end]
        elif strand_pat == -1:
            locs_dict_pat_negative[name_pat] = list(range(feature_pat.location.start, feature_pat.location.end))
            ref_genome_seq_patric_dict[name_pat] = genome_pat.seq[feature_pat.location.start:feature_pat.location.end].reverse_complement()

for feature_ref in genome_ref.features[:]:
    if feature_ref.type == 'CDS':
        name_ref = feature_ref.qualifiers['locus_tag'][0]
        strand_ref = feature_ref.strand
        if strand_ref == 1:
            locs_dict_ref_positive[name_ref] = list(range(feature_ref.location.start, feature_ref.location.end))
            ref_genome_seq_ref_dict[name_ref] = genome_ref.seq[feature_ref.location.start:feature_ref.location.end]

        elif strand_ref == -1:
            locs_dict_ref_negative[name_ref] = list(range(feature_ref.location.start, feature_ref.location.end))
            ref_genome_seq_ref_dict[name_ref] = genome_ref.seq[feature_ref.location.start:feature_ref.location.end].reverse_complement()
            
print(len(locs_dict_pat_positive.keys()))
print(len(locs_dict_pat_negative.keys()))
print(len(locs_dict_ref_positive.keys()))
print(len(locs_dict_ref_negative.keys()))   

reference_name_dict = {}
for name_pat, loc_pat in sorted(locs_dict_pat_positive.items()):
    for name_ref, loc_ref in sorted(locs_dict_ref_positive.items()):
        if len(set(loc_pat)&set(loc_ref)) / len(loc_pat) > 0.9:
            reference_name_dict[name_ref] = name_pat
            del locs_dict_ref_positive[name_ref]
            break
            
for name_pat, loc_pat in sorted(locs_dict_pat_negative.items()):
    for name_ref, loc_ref in sorted(locs_dict_ref_negative.items()):
        if len(set(loc_pat)&set(loc_ref)) / len(loc_pat) > 0.9:
            reference_name_dict[name_ref] = name_pat
            del locs_dict_ref_negative[name_ref]
            break
print(len(reference_name_dict.keys()), len(ref_genome_seq_patric_dict.keys()), len(ref_genome_seq_ref_dict.keys()))

### Load in dataset of protein abundance from PaxDB (integrated dataset)

In [ ]:
df = pd.read_csv('/Users/adamhockenberry/Downloads/511145/511145-WHOLE_ORGANISM-integrated.txt',\
                 index_col = 'string_external_id', sep='\t', skiprows=10)
expression_dict = {}
for index in df.index:
    temp_name = index.split('.')[-1]
    try:
        expression_dict[reference_name_dict[temp_name]] = df.loc[index]['abundance']
    except KeyError:
        pass

exp_threshold = np.percentile(list(expression_dict.values()), 50) #TUNABLE PARAMETER
expression_subset = []
for i,j in expression_dict.items():
    if j > exp_threshold:
        expression_subset.append(i)
print('Number of genes in the expression subset is {}'.format(len(expression_subset)))

### Get list of gene expression values in my dataset

In [ ]:
dict_to_analyze = dicty_hyphy

full_listy = []
for key in expression_dict.keys():
    if key in dict_to_analyze.keys():
        full_listy.append(expression_dict[key])
print(len(full_listy))

In [ ]:
percentile_20 = np.percentile(full_listy, 20)
percentile_40 = np.percentile(full_listy, 40)
percentile_60 = np.percentile(full_listy, 60)
percentile_80 = np.percentile(full_listy, 80)

expression_quantile_dict = {}
expression_quantile_dict['lowest'] = []
expression_quantile_dict['low'] = []
expression_quantile_dict['mid'] = []
expression_quantile_dict['high'] = []
expression_quantile_dict['highest'] = []

for seq_name in dict_to_analyze:
    if seq_name not in expression_dict.keys():
        continue
    if expression_dict[seq_name] < percentile_20:
        expression_quantile_dict['lowest'].append(seq_name)
    elif expression_dict[seq_name] >= percentile_20 and expression_dict[seq_name] < percentile_40:
        expression_quantile_dict['low'].append(seq_name)
    elif expression_dict[seq_name] >= percentile_40 and expression_dict[seq_name] < percentile_60:
        expression_quantile_dict['mid'].append(seq_name)
    elif expression_dict[seq_name] >= percentile_60 and expression_dict[seq_name] < percentile_80:
        expression_quantile_dict['high'].append(seq_name)
    elif expression_dict[seq_name] >= percentile_80 :
        expression_quantile_dict['highest'].append(seq_name)
    
for key, vals in expression_quantile_dict.items():
    print(key, len(vals))


In [ ]:
#####Parameters!
threshold=-4.5
only_conserved_aa=True
three_prime_analysis=False
controls=['CODON', 'TRINT']

meta_codon = []
meta_trint = []
for cat in ['lowest', 'low', 'mid', 'high', 'highest']:
    print('#####{}'.format(cat))
    subset_names = expression_quantile_dict[cat]
    actual_ratios_codon = []
    actual_ratios_trint = []
    for i in range(20):
        sds, others = paired_nt_analysis(dicty_to_analyze,\
                                     seq_dicty[base_genome],\
                                     fragment_energy_dict,\
                                     asd_seq,\
                                     conservation_dicty,\
                                     threshold=threshold,\
                                     randomization=False,\
                                     subset_names=subset_names,\
                                     only_conserved_aa=only_conserved_aa,\
                                     three_prime_analysis=three_prime_analysis,\
                                     internal_control=controls[0])
        actual_ratios_codon.append(np.mean(sds)/np.mean(others))

        sds, others = paired_nt_analysis(dicty_to_analyze,\
                             seq_dicty[base_genome],\
                             fragment_energy_dict,\
                             asd_seq,\
                             conservation_dicty,\
                             threshold=threshold,\
                             randomization=False,\
                             subset_names=subset_names,\
                             only_conserved_aa=only_conserved_aa,\
                             three_prime_analysis=three_prime_analysis,\
                             internal_control=controls[1])
        actual_ratios_trint.append(np.mean(sds)/np.mean(others))
        
    meta_codon.append(actual_ratios_codon)
    meta_trint.append(actual_ratios_trint)
    print(len(sds), np.mean(actual_ratios_codon), np.mean(actual_ratios_trint))

In [ ]:
len(meta_codon[0])

In [ ]:
plt.figure()
plt.violinplot(meta_codon, showextrema=False)


plt.figure()
plt.violinplot(meta_trint, showextrema=False)

# Codon specific fixed-effect

# Looking for the consensus nucleotide

In [ ]:
consensus_sd = []
consensus_other = []
sds = []
others = []
for gene_name, seq in list(seq_dicty[base_genome].items())[:]:
    if gene_name not in dicty_hyphy.keys():
        continue
    codon_list = [seq[nt:nt+3] for nt in range(0, len(seq), 3)]
    energy_list = []
    for i in range(0, len(seq)-len(asd_seq)):
        fragment = seq[i:i+len(asd_seq)]
        if fragment.count('-') == 0:
            energy = fragment_energy_dict[fragment.replace('T', 'U')]
            energy_list.append(energy)
        else:
            energy_list.append(np.nan)
    analyzed_positions = []
    sd_positions = [list(range(i, i+6)) for i,j in enumerate(energy_list) if j < threshold]
#     sd_positions = [list(range(i, i+6)) for i,j in enumerate(energy_list) if thresholdA < j < thresholdB]
    sd_positions = [item for sublist in sd_positions for item in sublist]
    sd_positions = list(set(sd_positions))
    for i, energy in enumerate(energy_list):
        modifiers = []
        if energy < threshold and i > 100 and i < len(seq)-50:
#         if thresholdA < energy < thresholdB and i > 100 and i < len(seq)-50:
#         if energy < threshold and i > 100 and i < len(seq)-50 and gene_name in expression_subset:
#         if energy < threshold and i > 50 and i < len(seq)-50 and gene_name in long_genes:
#         if energy < threshold and i > len(seq)-50 and gene_name in preceding_genes:
#         if energy < threshold and i > len(seq)-50 and gene_name not in preceding_genes:
            if i%3 == 0:
                modifiers = [2]
            elif i%3 == 2:
                modifiers = [3]
            elif i%3 == 1:
                modifiers = [1,4]
            for modifier in modifiers:
                if conservation_dicty[gene_name][int(np.floor((i+modifier)/3.))] != 'C':
                    continue
                codon = seq[i+modifier-2:i+modifier+1]
                if codon in redundant_codons:
                    fragment = seq[i:i+len(asd_seq)]
                    assert fragment_energy_dict[fragment.replace('T', 'U')] == energy
                    all_codons = aa_to_codon_dict[codon_to_aa_dict[codon]]
                    possible_energies = []
                    for possible_codon in all_codons:
                        if possible_codon != codon:
                            if modifier == 2:
                                possible_fragment = possible_codon + fragment[-3:]
                            elif modifier == 3:
                                possible_fragment = fragment[0] + possible_codon + fragment[-2:]
                            elif modifier == 1:
                                possible_fragment = possible_codon[-2:] + fragment[-4:]
                            elif modifier == 4:
                                possible_fragment = fragment[0:2] + possible_codon + fragment[-1]
                            assert len(possible_fragment) == 6
                            possible_energies.append(fragment_energy_dict[possible_fragment.replace('T', 'U')])

#                     if len(possible_energies) == 0:
#                         continue
#                     if energy > min(possible_energies):
#                         continue
                    if energy < max(possible_energies):
                        continue
#                     if energy > max(possible_energies) or energy < min(possible_energies):
#                         continue
                    control_sites = [j*3+2 for j, x in enumerate(codon_list) if x == codon]
                    control_sites = [j for j in control_sites if j not in sd_positions and j > 100 and j < len(seq)-50]
                    control_sites = [j for j in control_sites if conservation_dicty[gene_name][int(np.floor(j/3.))] == 'C']
                    control_sites = [j for j in control_sites if j not in analyzed_positions]
                    if len(control_sites) > 0 and i+modifier not in analyzed_positions:
                        choice = random.choice(control_sites)
#                         sds.append(dicty_phylop[gene_name][i+modifier])
#                         others.append(dicty_phylop[gene_name][choice])
                        sds.append(dicty_hyphy[gene_name][i+modifier])
                        others.append(dicty_hyphy[gene_name][choice])
                        analyzed_positions.append(i+modifier)
                        analyzed_positions.append(choice)
                        if nt_consensus_dicty[gene_name][i+modifier] == seq_dicty[base_genome][gene_name][i+modifier]:
                            consensus_sd.append(1)
                        else:
                            consensus_sd.append(0)
                            
                        if nt_consensus_dicty[gene_name][choice] == seq_dicty[base_genome][gene_name][choice]:
                            consensus_other.append(1)
                        else:
                            consensus_other.append(0)

In [ ]:
len(sds), len(others), len(consensus_sd), len(consensus_other)

In [ ]:
sum(consensus_sd), sum(consensus_other)

In [ ]:
84/93, 74/93

In [ ]:
222/326, 257/326

In [ ]:
307/394, 283/394

In [ ]:
1172/1845, 1360/1845

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
violin_parts = ax.violinplot([np.array(sds)-np.array(others), np.array(sds)-np.array(others)],\
                             vert=False, showextrema=False, showmeans=True, widths=0.75)
for pc in violin_parts['bodies']:
    pc.set_facecolor('gray')
    pc.set_edgecolor('gray')
    pc.set_alpha(0.55)
violin_parts['cmeans'].set_edgecolor('r')
ax.set_xlim(-1,1)


# ax.set_xlabel('Difference (SD - Control)');
# ax.set_ylabel('counts');
# ax.axvline(np.mean(np.array(sds)-np.array(others)), c='c', linewidth=2)
# plt.tight_layout()

In [ ]:
violin_parts['bodies']

In [ ]:
actual_rats = []
shuffled_rats = []
for i in range(100):
    sds, others = paired_analysis_trint_control(dicty_hyphy, seq_dicty[base_genome],\
                                            fragment_energy_dict, asd_seq, conservation_dicty,\
                                                subset_names = False,\
                                               randomization=False,\
                                               only_conserved_aa=False)
    actual_rats.append(np.mean(sds)/np.mean(others))
    print(i, len(sds))
    
#     sds, others = paired_analysis_codon_control(dicty_hyphy, seq_dicty[base_genome],\
#                                             fragment_energy_dict, asd_seq, conservation_dicty,\
#                                                randomization=True)
#     shuffled_rats.append(np.mean(sds)/np.mean(others))

In [ ]:
plt.figure()
plt.hist(actual_rats, alpha=0.5)
# plt.hist(shuffled_rats, alpha=0.5)

In [ ]:
print(stats.ttest_ind(actual_rats, shuffled_rats))